In [19]:
import requests
import pandas as pd
import plotly.express as px

cities = pd.read_csv('./places.csv').columns.to_list()

In [20]:
api_url = 'https://nominatim.openstreetmap.org/search'

enriched_cities = []

for city in cities:
    city = city.replace(' ', '-')           # cleaning city names so that we can pass them to the API with dashes instead of spaces

    payload = {'city' : city,
            'limit' : 1,
            'country' : 'france',
            'country-code' : 'fr',
            'format' : 'json'}
    
    info = requests.get(api_url, 
                        params=payload)
    
    enriched_cities.append({'city': city,
                            'lat' : float(info.json()[0]['lat']),
                            'lon' : float(info.json()[0]['lon'])})

In [21]:
api_keys = pd.read_csv('~/Documents/api_keys.csv')
api_url = 'https://api.openweathermap.org/data/2.5/forecast?'
web_url = api_url.split("/")[2]

key = api_keys.loc[api_keys['website'] == web_url, 'key'].iloc[0]

weather_data = []

for city in enriched_cities:
   cur_lat = str("%.2f" % city['lat'])
   cur_lon = str("%.2f" % city['lon'])
   req_url = api_url + 'lat=' + cur_lat + '&lon=' + cur_lon + '&units=metric' + '&appid=' + key

   info = requests.get(req_url)

   city['weather'] = info.json()             # we add a new 'weather' key to our dictionary


In [23]:
def weather_rating(df):
    df = df[df['sys.pod'] == 'd']              # we are only interested in the weather during the day
    avg_temp = df['main.feels_like'].mean()
    avg_pop = df['pop'].mean()
    avg_cloud = df['clouds.all'].mean()
    avg_wind = df['wind.speed'].mean()
    return (avg_temp, avg_pop, avg_cloud, avg_wind)

def my_rating(avg_temp, avg_pop, avg_cloud, avg_wind):
    return avg_temp - avg_pop*20 - avg_cloud/10 - avg_wind

In [24]:
df2 = pd.DataFrame(columns=["City", "Avg temp", "Avg pop", "Avg cloud", "Avg wind", "Rating", "lat", "lon"])
idx = 0

for city in enriched_cities:
    avg_temp, avg_pop, avg_cloud, avg_wind = weather_rating(pd.json_normalize(city['weather']['list']))
    rating = my_rating(avg_temp, avg_pop, avg_cloud, avg_wind)
    df2.loc[idx] = [city['city'], avg_temp, avg_pop, avg_cloud, avg_wind, rating, city['lat'], city['lon']]
    idx += 1

# correction to have only positive ratings so that we can use rating for dot size
# and also because it is easier to deal with scaled, positive ratings only from 0 to 10

rmin = df2['Rating'].min()
rmax = df2['Rating'].max()
df2['Rating'] = df2['Rating'].apply(lambda x : 10*(x-rmin)/(rmax-rmin))

display(df2.sort_values(by=['Rating'], ascending=False).head(10))

,City,Avg temp,Avg pop,Avg cloud,Avg wind,Rating,lat,lon
21,Aix-en-Provence,16.6520,0.0000,74.08,2.9628,10.000000,43.529842,5.447474
22,Avignon,16.5428,0.0236,69.48,2.9216,9.955054,43.949249,4.805901
29,Ariege,17.5752,0.0376,82.24,2.4232,9.940896,42.845802,1.606759
27,Collioure,16.6436,0.0000,81.28,3.2660,9.420423,42.525050,3.083155
24,Nimes,15.4776,0.0180,73.24,3.2880,9.002427,43.837425,4.360069
15,Grenoble,16.5832,0.1216,76.24,2.2036,8.900175,45.187560,5.735782
20,Marseille,15.7844,0.0000,71.04,4.7208,8.695672,43.296174,5.369953
33,Bayonne,17.1176,0.1612,74.44,2.9136,8.457683,43.494514,-1.473666
23,Uzes,15.3668,0.0840,75.52,2.9544,8.257899,44.012128,4.419672
25,Aigues-Mortes,14.6960,0.0000,70.84,4.5356,8.199470,43.566152,4.191540


In [26]:
fig = px.scatter_mapbox(df2,
                        lat='lat',
                        lon='lon',
                        color='Rating',
                        size='Rating',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        color_continuous_scale=['white', 'yellow', 'red'],
                        mapbox_style='open-street-map',
                        hover_name='City',
                        hover_data=['Avg temp', 'Avg pop', 'Avg cloud', 'Avg wind', 'Rating'],
                        height=600)

fig.show()


In [27]:
fig = px.scatter_mapbox(df2.sort_values(by=['Rating'], ascending=False).head(),
                        lat='lat',
                        lon='lon',
                        color='Rating',
                        size='Rating',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        color_continuous_scale=['yellow', 'orange', 'red'],
                        mapbox_style='open-street-map',
                        hover_name='City',
                        hover_data=['Avg temp', "Avg pop", "Avg cloud", "Avg wind", "Rating"],
                        height=600)

fig.show()

In [ ]:
df.to_csv('./weather_data.csv', index=False)